In [1]:
print("hello world")

hello world


In [4]:
import os
import shutil

# Define the directory to organize
source_dir ="C:\\Users\\admin\\OneDrive\\Desktop"

# Define file type categories and their extensions
file_types = {
    "Images": [".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff"],
    "Documents": [".pdf", ".doc", ".docx", ".txt", ".xls", ".xlsx", ".ppt", ".pptx"],
    "Videos": [".mp4", ".mov", ".avi", ".mkv", ".flv"],
    "Music": [".mp3", ".wav", ".aac", ".flac"],
    "Archives": [".zip", ".rar", ".tar", ".gz", ".7z"],
    "Scripts": [".py", ".js", ".html", ".css", ".sh", ".bat"],
    "Others": []
}

def organize_files(directory):
    # Create folders if they don't exist
    for folder in file_types.keys():
        folder_path = os.path.join(directory, folder)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    # Iterate over files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)

        # Skip directories
        if os.path.isdir(file_path):
            continue

        # Get file extension
        _, ext = os.path.splitext(filename)
        ext = ext.lower()

        # Find the category for the file
        moved = False
        for category, extensions in file_types.items():
            if ext in extensions:
                dest_path = os.path.join(directory, category, filename)
                shutil.move(file_path, dest_path)
                print(f"Moved {filename} to {category}")
                moved = True
                break

        # If no category matched, move to Others
        if not moved:
            dest_path = os.path.join(directory, "Others", filename)
            shutil.move(file_path, dest_path)
            print(f"Moved {filename} to Others")

if __name__ == "__main__":
    organize_files(source_dir)


Moved Assignment 2 (2).docx to Documents
Moved CodeBlocks.lnk to Others
Moved cursor.c to Others
Moved Cursor.lnk to Others
Moved CursorUserSetup-x64-0.49.6.exe to Others
Moved Decoderz (2).docx to Documents
Moved Microsoft Teams.lnk to Others
Moved PBL.pptx to Documents
Moved tj-Peace.py to Scripts
Moved W (1) (1).pptx to Documents


In [3]:
import os
import shutil
import json
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog

# Default file categories and extensions
default_file_types = {
    "Images": [".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff"],
    "Documents": [".pdf", ".doc", ".docx", ".txt", ".xls", ".xlsx", ".ppt", ".pptx"],
    "Videos": [".mp4", ".mov", ".avi", ".mkv", ".flv"],
    "Music": [".mp3", ".wav", ".aac", ".flac"],
    "Archives": [".zip", ".rar", ".tar", ".gz", ".7z"],
    "Scripts": [".py", ".js", ".html", ".css", ".sh", ".bat"],
    "Others": []
}

class JunkFileOrganizerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Junk File Organizer")

        self.source_dir = ""
        self.log_dir = ""  # Directory to save logs and config
        self.move_logs = []  # List of move log file paths for undo history
        self.file_types = default_file_types.copy()

        # GUI Elements
        self.label = tk.Label(root, text="Select a folder to organize junk files", font=("Arial", 14))
        self.label.pack(pady=10)

        self.select_btn = tk.Button(root, text="Select Folder", command=self.select_folder)
        self.select_btn.pack(pady=5)

        self.folder_label = tk.Label(root, text="No folder selected", fg="blue")
        self.folder_label.pack(pady=5)

        self.preview_btn = tk.Button(root, text="Preview Files", command=self.preview_files, state=tk.DISABLED)
        self.preview_btn.pack(pady=5)

        self.organize_btn = tk.Button(root, text="Organize Files", command=self.organize_files, state=tk.DISABLED)
        self.organize_btn.pack(pady=5)

        self.undo_btn = tk.Button(root, text="Undo Last Organize", command=self.undo_organize, state=tk.DISABLED)
        self.undo_btn.pack(pady=5)

        self.edit_categories_btn = tk.Button(root, text="Edit File Categories", command=self.edit_categories)
        self.edit_categories_btn.pack(pady=5)

        self.status_text = tk.Text(root, height=15, width=70, state=tk.DISABLED)
        self.status_text.pack(pady=10)

        # Preview window reference
        self.preview_window = None

    def log(self, message):
        self.status_text.config(state=tk.NORMAL)
        self.status_text.insert(tk.END, message + "\n")
        self.status_text.see(tk.END)
        self.status_text.config(state=tk.DISABLED)

    def select_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.source_dir = folder
            self.log_dir = os.path.join(self.source_dir, ".junk_organizer_logs")
            if not os.path.exists(self.log_dir):
                os.makedirs(self.log_dir)

            self.folder_label.config(text=self.source_dir)
            self.preview_btn.config(state=tk.NORMAL)
            self.organize_btn.config(state=tk.NORMAL)
            self.update_undo_button()
            self.log(f"Selected folder: {self.source_dir}")

    def update_undo_button(self):
        # Check if any move log files exist for undo
        logs = sorted(
            [f for f in os.listdir(self.log_dir) if f.startswith("file_move_log_") and f.endswith(".json")],
            reverse=True
        )
        self.move_logs = [os.path.join(self.log_dir, f) for f in logs]
        if self.move_logs:
            self.undo_btn.config(state=tk.NORMAL)
        else:
            self.undo_btn.config(state=tk.DISABLED)

    def get_category_for_extension(self, ext):
        ext = ext.lower()
        for category, extensions in self.file_types.items():
            if ext in extensions:
                return category
        return "Others"

    def preview_files(self):
        if not self.source_dir:
            messagebox.showwarning("Warning", "Please select a folder first.")
            return

        files = []
        for filename in os.listdir(self.source_dir):
            file_path = os.path.join(self.source_dir, filename)
            if os.path.isfile(file_path):
                _, ext = os.path.splitext(filename)
                category = self.get_category_for_extension(ext)
                files.append((filename, category))

        if self.preview_window and tk.Toplevel.winfo_exists(self.preview_window):
            self.preview_window.destroy()

        self.preview_window = tk.Toplevel(self.root)
        self.preview_window.title("Preview Files")

        tk.Label(self.preview_window, text="File Name", font=("Arial", 12, "bold"), width=40, anchor="w").grid(row=0, column=0, padx=5)
        tk.Label(self.preview_window, text="Category", font=("Arial", 12, "bold"), width=20, anchor="w").grid(row=0, column=1, padx=5)

        for i, (fname, cat) in enumerate(files, start=1):
            tk.Label(self.preview_window, text=fname, anchor="w", width=40).grid(row=i, column=0, sticky="w", padx=5)
            tk.Label(self.preview_window, text=cat, anchor="w", width=20).grid(row=i, column=1, sticky="w", padx=5)

        tk.Button(self.preview_window, text="Close", command=self.preview_window.destroy).grid(row=len(files)+1, column=0, columnspan=2, pady=10)

    def safe_move(self, src, dest):
        """
        Move file from src to dest.
        If dest exists, rename the file by adding a number suffix.
        """
        base, ext = os.path.splitext(dest)
        counter = 1
        new_dest = dest
        while os.path.exists(new_dest):
            new_dest = f"{base}({counter}){ext}"
            counter += 1
        shutil.move(src, new_dest)
        return new_dest

    def organize_files(self):
        if not self.source_dir:
            messagebox.showwarning("Warning", "Please select a folder first.")
            return

        # Create category folders
        for folder in self.file_types.keys():
            folder_path = os.path.join(self.source_dir, folder)
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

        move_log = {}

        try:
            for filename in os.listdir(self.source_dir):
                file_path = os.path.join(self.source_dir, filename)

                if os.path.isdir(file_path):
                    continue

                _, ext = os.path.splitext(filename)
                category = self.get_category_for_extension(ext)

                dest_folder = os.path.join(self.source_dir, category)
                dest_path = os.path.join(dest_folder, filename)

                new_dest_path = self.safe_move(file_path, dest_path)
                self.log(f"Moved {filename} to {category} as {os.path.basename(new_dest_path)}")
                move_log[new_dest_path] = file_path

            # Save move log with timestamp for undo history
            import datetime
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            log_path = os.path.join(self.log_dir, f"file_move_log_{timestamp}.json")
            with open(log_path, "w") as f:
                json.dump(move_log, f, indent=4)

            self.log("Organization complete.")
            self.update_undo_button()

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def undo_organize(self):
        if not self.move_logs:
            messagebox.showwarning("Warning", "No organization log found to undo.")
            return

        # Undo the most recent organize operation
        log_path = self.move_logs[0]

        try:
            with open(log_path, "r") as f:
                move_log = json.load(f)

            for new_path, original_path in move_log.items():
                if os.path.exists(new_path):
                    original_dir = os.path.dirname(original_path)
                    if not os.path.exists(original_dir):
                        os.makedirs(original_dir)
                    shutil.move(new_path, original_path)
                    self.log(f"Moved back {os.path.basename(new_path)} to original location")

            os.remove(log_path)
            self.log("Undo complete.")
            self.update_undo_button()

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def edit_categories(self):
        # Open a new window to edit categories and extensions
        edit_win = tk.Toplevel(self.root)
        edit_win.title("Edit File Categories")

        tk.Label(edit_win, text="Category", font=("Arial", 12, "bold"), width=20).grid(row=0, column=0, padx=5, pady=5)
        tk.Label(edit_win, text="Extensions (comma separated)", font=("Arial", 12, "bold"), width=40).grid(row=0, column=1, padx=5, pady=5)

        entries = {}

        # Show current categories and extensions
        for i, (category, extensions) in enumerate(self.file_types.items(), start=1):
            cat_label = tk.Label(edit_win, text=category, width=20, anchor="w")
            cat_label.grid(row=i, column=0, sticky="w", padx=5, pady=2)

            ext_str = ", ".join(extensions)
            ext_entry = tk.Entry(edit_win, width=50)
            ext_entry.insert(0, ext_str)
            ext_entry.grid(row=i, column=1, padx=5, pady=2)
            entries[category] = ext_entry

        def save_categories():
            new_file_types = {}
            for category, entry in entries.items():
                ext_text = entry.get().strip()
                if ext_text:
                    exts = [e.strip().lower() if e.strip().startswith('.') else '.' + e.strip().lower() for e in ext_text.split(",")]
                else:
                    exts = []
                new_file_types[category] = exts

            # Ensure 'Others' category exists and is empty
            if "Others" not in new_file_types:
                new_file_types["Others"] = []

            self.file_types = new_file_types
            self.log("File categories updated.")
            edit_win.destroy()

        save_btn = tk.Button(edit_win, text="Save", command=save_categories)
        save_btn.grid(row=len(self.file_types)+1, column=0, columnspan=2, pady=10)

def main():
    root = tk.Tk()
    app = JunkFileOrganizerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


In [1]:
import os
import shutil
import json
import tkinter as tk
from tkinter import filedialog, messagebox

file_types = {
    "Images": [".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff"],
    "Documents": [".pdf", ".doc", ".docx", ".txt", ".xls", ".xlsx", ".ppt", ".pptx"],
    "Videos": [".mp4", ".mov", ".avi", ".mkv", ".flv"],
    "Music": [".mp3", ".wav", ".aac", ".flac"],
    "Archives": [".zip", ".rar", ".tar", ".gz", ".7z"],
    "Scripts": [".py", ".js", ".html", ".css", ".sh", ".bat"],
    "Others": []
}

class JunkFileOrganizerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Junk File Organizer")
        self.root.geometry("600x400")
        self.root.configure(bg="#f0f4f8")

        self.source_dir = ""
        self.log_file = ""

        # Fonts and colors
        self.font_title = ("Segoe UI", 16, "bold")
        self.font_label = ("Segoe UI", 11)
        self.font_button = ("Segoe UI", 10, "bold")
        self.color_bg = "#f0f4f8"
        self.color_btn = "#4a90e2"
        self.color_btn_hover = "#357ABD"
        self.color_text = "#333333"
        self.color_status_bg = "#ffffff"

        # GUI Elements
        self.label = tk.Label(root, text="Select a folder to organize junk files", font=self.font_title, bg=self.color_bg, fg=self.color_text)
        self.label.pack(pady=(20, 10))

        self.select_btn = tk.Button(root, text="Select Folder", font=self.font_button, bg=self.color_btn, fg="white", activebackground=self.color_btn_hover, activeforeground="white", command=self.select_folder, relief="flat", padx=15, pady=7)
        self.select_btn.pack(pady=5)

        self.folder_label = tk.Label(root, text="No folder selected", font=self.font_label, bg=self.color_bg, fg="#666666")
        self.folder_label.pack(pady=5)

        self.organize_btn = tk.Button(root, text="Organize Files", font=self.font_button, bg=self.color_btn, fg="white", activebackground=self.color_btn_hover, activeforeground="white", command=self.organize_files, relief="flat", padx=15, pady=7, state=tk.DISABLED)
        self.organize_btn.pack(pady=5)

        self.undo_btn = tk.Button(root, text="Undo Last Organize", font=self.font_button, bg="#d9534f", fg="white", activebackground="#c9302c", activeforeground="white", command=self.undo_organize, relief="flat", padx=15, pady=7, state=tk.DISABLED)
        self.undo_btn.pack(pady=5)

        self.status_text = tk.Text(root, height=8, width=70, bg=self.color_status_bg, fg=self.color_text, font=self.font_label, state=tk.DISABLED, relief="sunken", bd=2)
        self.status_text.pack(pady=(15, 10), padx=10)

        # Button hover effects
        self.select_btn.bind("<Enter>", lambda e: self.select_btn.config(bg=self.color_btn_hover))
        self.select_btn.bind("<Leave>", lambda e: self.select_btn.config(bg=self.color_btn))

        self.organize_btn.bind("<Enter>", lambda e: self.organize_btn.config(bg=self.color_btn_hover))
        self.organize_btn.bind("<Leave>", lambda e: self.organize_btn.config(bg=self.color_btn))

        self.undo_btn.bind("<Enter>", lambda e: self.undo_btn.config(bg="#c9302c"))
        self.undo_btn.bind("<Leave>", lambda e: self.undo_btn.config(bg="#d9534f"))

    def log(self, message):
        self.status_text.config(state=tk.NORMAL)
        self.status_text.insert(tk.END, message + "\n")
        self.status_text.see(tk.END)
        self.status_text.config(state=tk.DISABLED)

    def select_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.source_dir = folder
            self.log_file = os.path.join(self.source_dir, "file_move_log.json")
            self.folder_label.config(text=self.source_dir, fg=self.color_text)
            self.organize_btn.config(state=tk.NORMAL)
            # Enable undo button only if log file exists
            if os.path.exists(self.log_file):
                self.undo_btn.config(state=tk.NORMAL)
            else:
                self.undo_btn.config(state=tk.DISABLED)
            self.log(f"Selected folder: {self.source_dir}")

    def organize_files(self):
        if not self.source_dir:
            messagebox.showwarning("Warning", "Please select a folder first.")
            return

        # Create folders
        for folder in file_types.keys():
            folder_path = os.path.join(self.source_dir, folder)
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

        move_log = {}

        try:
            for filename in os.listdir(self.source_dir):
                file_path = os.path.join(self.source_dir, filename)

                if os.path.isdir(file_path):
                    continue

                _, ext = os.path.splitext(filename)
                ext = ext.lower()

                moved = False
                for category, extensions in file_types.items():
                    if ext in extensions:
                        dest_path = os.path.join(self.source_dir, category, filename)
                        shutil.move(file_path, dest_path)
                        self.log(f"Moved {filename} to {category}")
                        move_log[dest_path] = file_path
                        moved = True
                        break

                if not moved:
                    dest_path = os.path.join(self.source_dir, "Others", filename)
                    shutil.move(file_path, dest_path)
                    self.log(f"Moved {filename} to Others")
                    move_log[dest_path] = file_path

            # Save move log
            with open(self.log_file, "w") as f:
                json.dump(move_log, f, indent=4)

            self.log("Organization complete.")
            self.undo_btn.config(state=tk.NORMAL)

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def undo_organize(self):
        if not self.source_dir or not os.path.exists(self.log_file):
            messagebox.showwarning("Warning", "No organization log found to undo.")
            return

        try:
            with open(self.log_file, "r") as f:
                move_log = json.load(f)

            for new_path, original_path in move_log.items():
                if os.path.exists(new_path):
                    original_dir = os.path.dirname(original_path)
                    if not os.path.exists(original_dir):
                        os.makedirs(original_dir)
                    shutil.move(new_path, original_path)
                    self.log(f"Moved back {os.path.basename(new_path)} to original location")

            os.remove(self.log_file)
            self.log("Undo complete.")
            self.undo_btn.config(state=tk.DISABLED)

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

def main():
    root = tk.Tk()
    app = JunkFileOrganizerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


In [ ]:
import tkinter as tk
from tkinter import messagebox

class Checkers:
    def __init__(self, root):
        self.root = root
        self.root.title("Checkers Game")
        self.board_size = 8
        self.cell_size = 60
        self.canvas = tk.Canvas(root, width=self.board_size*self.cell_size,
                                height=self.board_size*self.cell_size)
        self.canvas.pack()
        
        self.selected_piece = None
        self.turn = 'red'  # red starts
        self.pieces = {}  # (row, col): {'color': 'red'/'black', 'king': bool}
        
        self.draw_board()
        self.init_pieces()
        self.draw_pieces()
        
        self.canvas.bind("<Button-1>", self.click)

    def draw_board(self):
        self.canvas.delete("square")
        color1 = "#DDB88C"
        color2 = "#A66D4F"
        for row in range(self.board_size):
            for col in range(self.board_size):
                color = color1 if (row + col) % 2 == 0 else color2
                x1 = col * self.cell_size
                y1 = row * self.cell_size
                x2 = x1 + self.cell_size
                y2 = y1 + self.cell_size
                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color, tags="square")

    def init_pieces(self):
        # Place black pieces on rows 0,1,2 on dark squares
        for row in range(3):
            for col in range(self.board_size):
                if (row + col) % 2 == 1:
                    self.pieces[(row, col)] = {'color': 'black', 'king': False}
        # Place red pieces on rows 5,6,7 on dark squares
        for row in range(5, 8):
            for col in range(self.board_size):
                if (row + col) % 2 == 1:
                    self.pieces[(row, col)] = {'color': 'red', 'king': False}

    def draw_pieces(self):
        self.canvas.delete("piece")
        for (row, col), piece in self.pieces.items():
            x = col * self.cell_size + self.cell_size // 2
            y = row * self.cell_size + self.cell_size // 2
            radius = self.cell_size // 2 - 5
            fill_color = piece['color']
            self.canvas.create_oval(x - radius, y - radius, x + radius, y + radius,
                                    fill=fill_color, tags="piece")
            if piece['king']:
                # Draw a crown or mark for king
                self.canvas.create_text(x, y, text="K", fill="white", font=("Arial", 24, "bold"), tags="piece")

    def click(self, event):
        col = event.x // self.cell_size
        row = event.y // self.cell_size
        if (row, col) not in self.pieces and self.selected_piece is None:
            # Clicked empty square without selecting a piece
            return
        if self.selected_piece is None:
            # Select a piece if it belongs to current player
            if (row, col) in self.pieces and self.pieces[(row, col)]['color'] == self.turn:
                self.selected_piece = (row, col)
                self.highlight_moves()
        else:
            # Try to move selected piece to clicked square
            if (row, col) == self.selected_piece:
                # Deselect
                self.selected_piece = None
                self.draw_board()
                self.draw_pieces()
                return
            if self.is_valid_move(self.selected_piece, (row, col)):
                self.make_move(self.selected_piece, (row, col))
                self.selected_piece = None
                self.draw_board()
                self.draw_pieces()
                self.switch_turn()
                if self.check_winner():
                    return
            else:
                # Invalid move, deselect
                self.selected_piece = None
                self.draw_board()
                self.draw_pieces()

    def highlight_moves(self):
        self.draw_board()
        self.draw_pieces()
        moves = self.get_valid_moves(self.selected_piece)
        for move in moves:
            row, col = move
            x1 = col * self.cell_size
            y1 = row * self.cell_size
            x2 = x1 + self.cell_size
            y2 = y1 + self.cell_size
            self.canvas.create_rectangle(x1, y1, x2, y2, outline="blue", width=3, tags="highlight")

    def get_valid_moves(self, pos):
        row, col = pos
        piece = self.pieces[pos]
        directions = []
        if piece['color'] == 'red' or piece['king']:
            directions.append((-1, -1))
            directions.append((-1, 1))
        if piece['color'] == 'black' or piece['king']:
            directions.append((1, -1))
            directions.append((1, 1))

        valid_moves = []
        # Normal moves
        for dr, dc in directions:
            r, c = row + dr, col + dc
            if 0 <= r < self.board_size and 0 <= c < self.board_size:
                if (r, c) not in self.pieces:
                    valid_moves.append((r, c))

        # Capture moves
        for dr, dc in directions:
            r1, c1 = row + dr, col + dc
            r2, c2 = row + 2*dr, col + 2*dc
            if 0 <= r2 < self.board_size and 0 <= c2 < self.board_size:
                if (r1, c1) in self.pieces and self.pieces[(r1, c1)]['color'] != piece['color']:
                    if (r2, c2) not in self.pieces:
                        valid_moves.append((r2, c2))

        return valid_moves

    def is_valid_move(self, start, end):
        valid_moves = self.get_valid_moves(start)
        return end in valid_moves

    def make_move(self, start, end):
        piece = self.pieces[start]
        del self.pieces[start]
        self.pieces[end] = piece

        # Check if move is a capture
        r1, c1 = start
        r2, c2 = end
        if abs(r2 - r1) == 2:
            # Remove captured piece
            captured = ((r1 + r2)//2, (c1 + c2)//2)
            if captured in self.pieces:
                del self.pieces[captured]

        # Check for king promotion
        if piece['color'] == 'red' and r2 == 0:
            piece['king'] = True
        elif piece['color'] == 'black' and r2 == self.board_size - 1:
            piece['king'] = True

    def switch_turn(self):
        self.turn = 'black' if self.turn == 'red' else 'red'
        self.root.title(f"Checkers Game - {self.turn.capitalize()}'s Turn")

    def check_winner(self):
        red_pieces = [p for p in self.pieces.values() if p['color'] == 'red']
        black_pieces = [p for p in self.pieces.values() if p['color'] == 'black']
        if not red_pieces:
            messagebox.showinfo("Game Over", "Black wins!")
            self.root.destroy()
            return True
        elif not black_pieces:
            messagebox.showinfo("Game Over", "Red wins!")
            self.root.destroy()
            return True
        return False

if __name__ == "__main__":
    root = tk.Tk()
    game = Checkers(root)
    root.mainloop()


In [4]:
import tkinter as tk
from tkinter import messagebox
import random

class Checkers:
    def __init__(self, root):
        self.root = root
        self.root.title("Checkers Game - Your turn (Red)")
        self.board_size = 8
        self.cell_size = 60
        self.canvas = tk.Canvas(root, width=self.board_size*self.cell_size,
                                height=self.board_size*self.cell_size)
        self.canvas.pack()
        
        self.selected_piece = None
        self.turn = 'red'  # red (human) starts
        self.pieces = {}  # (row, col): {'color': 'red'/'black', 'king': bool}
        
        self.draw_board()
        self.init_pieces()
        self.draw_pieces()
        
        self.canvas.bind("<Button-1>", self.click)

    def draw_board(self):
        self.canvas.delete("square")
        color1 = "#DDB88C"
        color2 = "#A66D4F"
        for row in range(self.board_size):
            for col in range(self.board_size):
                color = color1 if (row + col) % 2 == 0 else color2
                x1 = col * self.cell_size
                y1 = row * self.cell_size
                x2 = x1 + self.cell_size
                y2 = y1 + self.cell_size
                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color, tags="square")

    def init_pieces(self):
        for row in range(3):
            for col in range(self.board_size):
                if (row + col) % 2 == 1:
                    self.pieces[(row, col)] = {'color': 'black', 'king': False}
        for row in range(5, 8):
            for col in range(self.board_size):
                if (row + col) % 2 == 1:
                    self.pieces[(row, col)] = {'color': 'red', 'king': False}

    def draw_pieces(self):
        self.canvas.delete("piece")
        for (row, col), piece in self.pieces.items():
            x = col * self.cell_size + self.cell_size // 2
            y = row * self.cell_size + self.cell_size // 2
            radius = self.cell_size // 2 - 5
            fill_color = piece['color']
            self.canvas.create_oval(x - radius, y - radius, x + radius, y + radius,
                                    fill=fill_color, tags="piece")
            if piece['king']:
                self.canvas.create_text(x, y, text="K", fill="white", font=("Arial", 24, "bold"), tags="piece")

    def click(self, event):
        if self.turn != 'red':
            return  # Ignore clicks when it's AI's turn

        col = event.x // self.cell_size
        row = event.y // self.cell_size
        if (row, col) not in self.pieces and self.selected_piece is None:
            return
        if self.selected_piece is None:
            if (row, col) in self.pieces and self.pieces[(row, col)]['color'] == self.turn:
                self.selected_piece = (row, col)
                self.highlight_moves()
        else:
            if (row, col) == self.selected_piece:
                self.selected_piece = None
                self.draw_board()
                self.draw_pieces()
                return
            if self.is_valid_move(self.selected_piece, (row, col)):
                self.make_move(self.selected_piece, (row, col))
                self.selected_piece = None
                self.draw_board()
                self.draw_pieces()
                if self.check_winner():
                    return
                self.switch_turn()
                self.root.after(500, self.ai_move)  # AI moves after 0.5 sec
            else:
                self.selected_piece = None
                self.draw_board()
                self.draw_pieces()

    def highlight_moves(self):
        self.draw_board()
        self.draw_pieces()
        moves = self.get_valid_moves(self.selected_piece)
        for move in moves:
            row, col = move
            x1 = col * self.cell_size
            y1 = row * self.cell_size
            x2 = x1 + self.cell_size
            y2 = y1 + self.cell_size
            self.canvas.create_rectangle(x1, y1, x2, y2, outline="blue", width=3, tags="highlight")

    def get_valid_moves(self, pos):
        row, col = pos
        piece = self.pieces[pos]
        directions = []
        if piece['color'] == 'red' or piece['king']:
            directions.append((-1, -1))
            directions.append((-1, 1))
        if piece['color'] == 'black' or piece['king']:
            directions.append((1, -1))
            directions.append((1, 1))

        valid_moves = []
        # Normal moves
        for dr, dc in directions:
            r, c = row + dr, col + dc
            if 0 <= r < self.board_size and 0 <= c < self.board_size:
                if (r, c) not in self.pieces:
                    valid_moves.append((r, c))

        # Capture moves
        for dr, dc in directions:
            r1, c1 = row + dr, col + dc
            r2, c2 = row + 2*dr, col + 2*dc
            if 0 <= r2 < self.board_size and 0 <= c2 < self.board_size:
                if (r1, c1) in self.pieces and self.pieces[(r1, c1)]['color'] != piece['color']:
                    if (r2, c2) not in self.pieces:
                        valid_moves.append((r2, c2))

        return valid_moves

    def is_valid_move(self, start, end):
        valid_moves = self.get_valid_moves(start)
        return end in valid_moves

    def make_move(self, start, end):
        piece = self.pieces[start]
        del self.pieces[start]
        self.pieces[end] = piece

        r1, c1 = start
        r2, c2 = end
        if abs(r2 - r1) == 2:
            captured = ((r1 + r2)//2, (c1 + c2)//2)
            if captured in self.pieces:
                del self.pieces[captured]

        # King promotion
        if piece['color'] == 'red' and r2 == 0:
            piece['king'] = True
        elif piece['color'] == 'black' and r2 == self.board_size - 1:
            piece['king'] = True

    def switch_turn(self):
        self.turn = 'black' if self.turn == 'red' else 'red'
        self.root.title(f"Checkers Game - {self.turn.capitalize()}'s Turn")

    def check_winner(self):
        red_pieces = [p for p in self.pieces.values() if p['color'] == 'red']
        black_pieces = [p for p in self.pieces.values() if p['color'] == 'black']
        if not red_pieces:
            messagebox.showinfo("Game Over", "Black (AI) wins!")
            self.root.destroy()
            return True
        elif not black_pieces:
            messagebox.showinfo("Game Over", "Red (You) wins!")
            self.root.destroy()
            return True
        return False

    def ai_move(self):
        if self.turn != 'black':
            return

        all_moves = self.get_all_moves('black')
        if not all_moves:
            messagebox.showinfo("Game Over", "Red (You) wins! AI has no moves.")
            self.root.destroy()
            return

        # Prefer capture moves
        capture_moves = [m for m in all_moves if abs(m[0][0] - m[1][0]) == 2]
        if capture_moves:
            move = random.choice(capture_moves)
        else:
            move = random.choice(all_moves)

        start, end = move
        self.make_move(start, end)
        self.draw_board()
        self.draw_pieces()
        if self.check_winner():
            return
        self.switch_turn()

    def get_all_moves(self, color):
        moves = []
        for pos, piece in self.pieces.items():
            if piece['color'] == color:
                valid_moves = self.get_valid_moves(pos)
                for move in valid_moves:
                    moves.append((pos, move))
        return moves

if __name__ == "__main__":
    root = tk.Tk()
    game = Checkers(root)
    root.mainloop()


In [1]:
import tkinter as tk
from tkinter import messagebox
import random

class TicTacToe:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("Tic-Tac-Toe: Human (X) vs AI (O)")
        self.window.geometry("300x300")
        
        # Board state: 3x3 grid, empty=' ', X='X', O='O'
        self.board = [[' ' for _ in range(3)] for _ in range(3)]
        
        # Current player: True for Human (X), False for AI (O)
        self.human_turn = True
        
        # Create buttons for the board
        self.buttons = []
        for i in range(3):
            row = []
            for j in range(3):
                btn = tk.Button(self.window, text=' ', font=('Arial', 20), width=5, height=2,
                                command=lambda row=i, col=j: self.make_move(row, col))
                btn.grid(row=i, column=j)
                row.append(btn)
            self.buttons.append(row)
        
        # Start the game with Human's turn
        self.reset_game()
    
    def reset_game(self):
        self.board = [[' ' for _ in range(3)] for _ in range(3)]
        self.human_turn = True
        for i in range(3):
            for j in range(3):
                self.buttons[i][j].config(text=' ', state='normal')
    
    def make_move(self, row, col):
        if self.board[row][col] != ' ' or not self.human_turn:
            return  # Invalid move
        
        # Human move (X)
        self.board[row][col] = 'X'
        self.buttons[row][col].config(text='X', state='disabled')
        
        if self.check_winner('X'):
            messagebox.showinfo("Game Over", "Human (X) Wins!")
            return
        
        if self.is_full():
            messagebox.showinfo("Game Over", "It's a Tie!")
            return
        
        # Switch to AI turn
        self.human_turn = False
        self.window.after(500, self.ai_move)  # Delay for better UX
    
    def ai_move(self):
        # AI (O) chooses best move using minimax
        best_score = -float('inf')
        best_move = None
        
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == ' ':
                    self.board[i][j] = 'O'
                    score = self.minimax(self.board, 0, False)
                    self.board[i][j] = ' '
                    if score > best_score:
                        best_score = score
                        best_move = (i, j)
        
        if best_move:
            row, col = best_move
            self.board[row][col] = 'O'
            self.buttons[row][col].config(text='O', state='disabled')
            
            if self.check_winner('O'):
                messagebox.showinfo("Game Over", "AI (O) Wins!")
                return
            
            if self.is_full():
                messagebox.showinfo("Game Over", "It's a Tie!")
                return
        
        # Switch back to Human turn
        self.human_turn = True
    
    def minimax(self, board, depth, is_maximizing):
        # Base cases
        if self.check_winner('O'):  # AI wins
            return 10 - depth
        if self.check_winner('X'):  # Human wins
            return depth - 10
        if self.is_full(board):
            return 0
        
        if is_maximizing:  # AI's turn (maximize)
            max_eval = -float('inf')
            for i in range(3):
                for j in range(3):
                    if board[i][j] == ' ':
                        board[i][j] = 'O'
                        eval_score = self.minimax(board, depth + 1, False)
                        board[i][j] = ' '
                        max_eval = max(max_eval, eval_score)
            return max_eval
        else:  # Human's turn (minimize)
            min_eval = float('inf')
            for i in range(3):
                for j in range(3):
                    if board[i][j] == ' ':
                        board[i][j] = 'X'
                        eval_score = self.minimax(board, depth + 1, True)
                        board[i][j] = ' '
                        min_eval = min(min_eval, eval_score)
            return min_eval
    
    def check_winner(self, player):
        # Check rows, columns, diagonals
        # Rows
        for i in range(3):
            if all(self.board[i][j] == player for j in range(3)):
                return True
        # Columns
        for j in range(3):
            if all(self.board[i][j] == player for i in range(3)):
                return True
        # Diagonals
        if all(self.board[i][i] == player for i in range(3)):
            return True
        if all(self.board[i][2-i] == player for i in range(3)):
            return True
        return False
    
    def is_full(self, board=None):
        if board is None:
            board = self.board
        return all(all(cell != ' ' for cell in row) for row in board)
    
    def run(self):
        self.window.mainloop()

# Run the game
if __name__ == "__main__":
    game = TicTacToe()
    game.run()
